SentenceTransformers: https://sbert.net/

# Load Data (Without Reviews)

In [ ]:
import pandas as pd
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/totallymakescents/fra_standard.csv'
with open(file_path, encoding='latin1') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        print(f"Line {i+1}: {row}")
        if i >= 9:  # print first 10 lines
            break

In [ ]:
df = pd.read_csv(
    file_path,
    sep=',',            # specify correct separator
    encoding='latin1',
    low_memory=False
)

In [ ]:
df.columns

In [ ]:
print(f"Shape: {df.shape}")  # (rows, columns)

In [ ]:
df.head(10)

# Load Data (with Reviews)


In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/totallymakescents/perfumes_table.csv'
with open(file_path, encoding='latin1') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        print(f"Line {i+1}: {row}")
        if i >= 9:  # print first 10 lines
            break

In [ ]:
df_review = pd.read_csv(
    file_path,
    sep=',',            # specify correct separator
    encoding='latin1',
    low_memory=False
)

In [ ]:
df_review.columns

In [ ]:
print(f"Shape: {df_review.shape}")  # (rows, columns)

In [ ]:
df_review.head(10)

# Merge Data

In [ ]:
df['url'] = df['url'].str.strip().str.lower()
df_review['url'] = df_review['url'].str.strip().str.lower()

# remove trailing "/"
df['url'] = df['url'].str.rstrip('/')
df_review['url'] = df_review['url'].str.rstrip('/')

In [ ]:
# concatinate all non-nan reviews into a single string
agg_reviews = df_review.groupby('url')['reviews'].apply(lambda x: ' '.join(x.dropna().astype(str))).reset_index()

combined_df = pd.merge(df, agg_reviews, on='url', how='left')

In [ ]:
print(combined_df.columns)
print(combined_df.shape)
combined_df.head()

In [ ]:
non_null_reviews = combined_df['reviews'].notnull().sum()
print(f"Perfumes with reviews: {non_null_reviews} / {len(combined_df)}")

# Sentence-BERT

In [ ]:
pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
# load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# select note columns
note_cols = ['Top', 'Middle', 'Base',
             'mainaccord1', 'mainaccord2', 'mainaccord3',
             'mainaccord4', 'mainaccord5']

# fill NaNs and combine into a string
# df['scent_description'] = df[note_cols].fillna('').agg(' '.join, axis=1)

In [ ]:
combined_df['reviews'] = combined_df['reviews'].str.replace(r'^\[?"|"?\]$', '', regex=True)

In [ ]:
# def build_full_description(row):
#     scent_text = ' '.join([str(row.get(col, '')) for col in note_cols])
#     review_text = str(row.get('reviews', ''))
#     return (scent_text + ' ' + review_text).strip()

def build_full_description(row):
    # so that BERT can understand different fields

    top = row.get('Top', '')
    middle = row.get('Middle', '')
    base = row.get('Base', '')

    accords = ', '.join([
        str(row.get('mainaccord1', '')),
        str(row.get('mainaccord2', '')),
        str(row.get('mainaccord3', '')),
        str(row.get('mainaccord4', '')),
        str(row.get('mainaccord5', ''))
    ])

    reviews = row.get('reviews', '')

    description = (
        f"Top Notes: {top}. "
        f"Middle Notes: {middle}. "
        f"Base Notes: {base}. "
        f"Main Accords: {accords}. "
        f"User Review: {reviews}"
    )

    return description.strip()

combined_df['full_description'] = combined_df.apply(build_full_description, axis=1)

In [ ]:
combined_df.columns

In [ ]:
# see the change
combined_df[['Top', 'Middle', 'Base',
    'mainaccord1', 'mainaccord2', 'mainaccord3',
    'mainaccord4', 'mainaccord5', 'full_description']].head(5)

In [ ]:
print(combined_df['full_description'].iloc[0])

In [ ]:
# now we are ready to build embeddings

scent_lists_with_review = combined_df['full_description'].tolist()
scent_embeddings = model.encode(
    scent_lists_with_review,
    batch_size=32,
    convert_to_tensor=True,
    show_progress_bar=True
)

In [ ]:
def recommend_perfumes(user_query, top_k=5):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, scent_embeddings)[0]
    # print(similarities)
    top_results = torch.topk(similarities, k=top_k)

    print(f"\nUser Query: {user_query}\n")
    for score, idx in zip(top_results.values, top_results.indices):
        idx = idx.item()        # convert Python tensor to int
        perfume = df.iloc[idx]
        print(f"{perfume['Perfume']} by {perfume['Brand']} (Score: {score.item():.3f})")
        print(f"Notes: {perfume['scent_description']}\n")

In [ ]:
recommend_perfumes("A magical scent in a mystical forest with herbs and secrets")

In [ ]:
recommend_perfumes("What’s it smell like in the rain, at the end of a hiking trail full of blossoms?")

In [ ]:
recommend_perfumes("What fragrance would a wizard wear in a magical world?")

In [ ]:
recommend_perfumes("Looking for a bittersweet scent for a farewell party.")

# LLM Generates Explanation
Follows this notebook:
https://colab.research.google.com/drive/1T5-zKWM_5OD21QHwXHiV9ixTRR7k3iB9?usp=sharing

In [ ]:
# Chat-GPT generated data, will replace later with data from Hugging Face
# to train LLM to generate meaningful explanation, need more data ~100-200, will add more
in_out_data = [
    {
    "input": "User wants a scent for a farewell party. Scent: citrus, violet, ambergris.",
    "output": "This perfume evokes a bittersweet goodbye — bright citrus for the goodbye, softened by violet and grounded in ambergris like a fading memory."
    },
    {
    "input": "User seeks a scent for a first date. Scent: raspberry, rose, sandalwood.",
    "output": "Flirty and romantic — sweet raspberry teases at the top, while rose adds elegance, and sandalwood brings warm confidence."
    },
    {
    "input": "User asks for a scent that feels like winter morning in a snowy cabin. Scent: pine, incense, vanilla.",
    "output": "This scent blends cool pine with smoky incense and cozy vanilla — a quiet snow-covered morning by the fire."
    },
    {
    "input": "User wants something that smells like a library filled with old books. Scent: leather, cedarwood, patchouli.",
    "output": "Evoking antique leather and aged wood, this scent conjures dusty pages, quiet corners, and timeless thought."
    },
    {
      "input": "User needs a scent for celebrating a big achievement. Scent: champagne accord, white florals, musk.",
      "output": "A sparkling celebration — fizzy champagne and bright florals rise with joy, grounded by elegant musk."
    }
]

In [ ]:
import json

with open("perfume_reasoning_dataset.jsonl", "w") as f:
    for example in in_out_data:
        json_line = json.dumps(example)
        f.write(json_line + "\n")


## Install Unsloth

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

## Data Prep

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("json", data_files="perfume_reasoning_dataset.jsonl", split="train")

In [ ]:
# needs this format when fine-tuning LLaMA3
def formatting_prompts_func(example):
    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{example['input']}<|eot_id|>\n" \
             f"<|start_header_id|>assistant<|end_header_id|>\n\n{example['output']}<|eot_id|>"
    return {"text": prompt}

dataset = dataset.map(formatting_prompts_func)

In [ ]:
print(dataset[4]["text"])

## Finetune Model with LLama3.2-Unsloth

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
# Only training on assistant responses

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
# Verifying data format

tokenizer.decode(trainer.train_dataset[4]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[4]["labels"]])

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## Run the model

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "role": "user",
        "content": "User wants a scent for a farewell party. Scent: citrus, violet, ambergris.",
    },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

In [ ]:
# Use a TextStreamer for continuous inference

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {
        "role": "user",
        "content": "User wants a scent for a graduation ceremony. Scent: citrus, violet, ambergris.",
    },
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

In [ ]:
# overfitting because only has 5 data set in the in_out_data, needs to add more data
# see a way for users to input the next prompt
# use the output from sentence-bert as an input for LLM